# Complex Urban Dataset
- Complex Urban Dataset with Multi-level Sensors from Highly Diverse Urban Environments (IJRR 2019)
- https://sites.google.com/view/complex-urban-dataset

## Import

In [33]:
import os
from io import TextIOWrapper
import sys
import struct
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation
from pathlib import Path
from glob import glob
import tarfile

### Before extraction
```bash
dataset_kaist_urban/
├── campus00
│   ├── campus00_calibration.tar.gz
│   ├── campus00_data.tar.gz
│   ├── campus00_las.tar.gz
│   └── campus00_pose.tar.gz
├── urban00
│   ├── urban00_calibration.tar.gz
│   ├── urban00_data.tar.gz
│   ├── urban00_las.tar.gz
│   └── urban00_pose.tar.gz
├── urban01
├── urban02
├── urban03
└── urban04
```


## Functions

In [37]:
def extract_tar(root: Path, tar_fn: Path):
    with tarfile.open(tar_fn) as tar :
        tar.extractall(path=root)

def rotmat2quat(R):
    rotation = Rotation.from_matrix(R)
    quaternion = rotation.as_quat()
    return quaternion

def procPose(root: Path, seq: str):
    old_pose_fn = root / seq / 'global_pose.csv'
    new_pose_fn = root / seq / 'pose.txt'

    f_old = open(old_pose_fn, 'r')
    f_new = open(new_pose_fn, 'w')

    while True:
        line = f_old.readline()
        if not line:
            break
        rot = np.zeros((3, 3), dtype=float)
        tra = np.zeros((3,), dtype=float)
        words = line.split(',')
        timestamp = float(words[0]) / 1e9
        rot[0,0] = float(words[1])
        rot[0,1] = float(words[2])
        rot[0,2] = float(words[3])
        tra[0]   = float(words[4])
        rot[1,0] = float(words[5])
        rot[1,1] = float(words[6])
        rot[1,2] = float(words[7])
        tra[1]   = float(words[8])
        rot[2,0] = float(words[9])
        rot[2,1] = float(words[10])
        rot[2,2] = float(words[11])
        tra[2]   = float(words[12])
        quat = rotmat2quat(rot)
        new_words = []
        new_words.append(f'{timestamp:.6f}')
        new_words.append(f'{tra[0]:.6f}')
        new_words.append(f'{tra[1]:.6f}')
        new_words.append(f'{tra[2]:.6f}')
        new_words.append(f'{quat[0]:.6f}')
        new_words.append(f'{quat[1]:.6f}')
        new_words.append(f'{quat[2]:.6f}')
        new_words.append(f'{quat[3]:.6f}')
        new_line = ' '.join(new_words) + '\n'
        f_new.write(new_line)
        # print(timestamp)
        # print(f"quat: [{quat[0]:.3f} {quat[1]:.3f} {quat[2]:.3f} {quat[3]:.3f}]")
        # print(f"tran: [{tra[0]:.3f} {tra[1]:.3f} {tra[2]:.3f}]")

    f_old.close()
    f_new.close()
    print(f"- done pose.txt")

def procCalib(root: Path, seq: str, old_name: str, new_name: str):
    old_calib_fn = root / seq / 'calibration' / old_name
    new_calib_fn = root / seq / 'calibration' / new_name

    f_old = open(old_calib_fn, 'r')
    f_new = open(new_calib_fn, 'w')
    
    lines = f_old.readlines()
    R_words = lines[3][2:].strip().split(' ')
    t_words = lines[4][2:].strip().split(' ')
    rotmat = np.zeros((3, 3), dtype=float)
    trans = np.zeros((3,), dtype=float)
    rotmat[0,0] = float(R_words[0])
    rotmat[0,1] = float(R_words[1])
    rotmat[0,2] = float(R_words[2])
    rotmat[1,0] = float(R_words[3])
    rotmat[1,1] = float(R_words[4])
    rotmat[1,2] = float(R_words[5])
    rotmat[2,0] = float(R_words[6])
    rotmat[2,1] = float(R_words[7])
    rotmat[2,2] = float(R_words[8])
    trans[0]   = float(t_words[0])
    trans[1]   = float(t_words[1])
    trans[2]   = float(t_words[2])
    quat = rotmat2quat(rotmat)
    new_words = []
    new_words.append(f'{trans[0]:.6f}')
    new_words.append(f'{trans[1]:.6f}')
    new_words.append(f'{trans[2]:.6f}')
    new_words.append(f'{quat[0]:.6f}')
    new_words.append(f'{quat[1]:.6f}')
    new_words.append(f'{quat[2]:.6f}')
    new_words.append(f'{quat[3]:.6f}')
    new_line = ' '.join(new_words) + '\n'
    
    f_new.write(new_line)
    print(new_line, end='')
    
    f_old.close()
    f_new.close()
    print(f"- done {new_name}")
    
def procLidarScans(root: Path, seq: str):
    for pos in ['left', 'right']:
        
        old_data_fn = root / seq / 'sensor_data' / f'VLP_{pos}_stamp.csv'
        new_data_fn = root / seq / f'lidar_{pos}_data.txt'
        f_old_data = open(old_data_fn, 'r')
        f_new_data = open(new_data_fn, 'w')
        
        old_lidar_dir = root / seq / 'sensor_data' / f'VLP_{pos}'
        new_lidar_dir = root / seq / f'lidar_{pos}'
        os.makedirs(new_lidar_dir, exist_ok=True)

        idx = 0
        while True:
            line = f_old_data.readline().strip()
            if not line: 
                break
            timestamp = float(line) / 1e9
            
            old_lidar_path = old_lidar_dir / f"{line}.bin"
            new_lidar_path = new_lidar_dir / f"{idx:06d}.bin"
            if not old_lidar_path.exists():
                print(f"{old_lidar_path} doesn't exist!")
                continue
            os.rename(
                old_lidar_path,
                new_lidar_path)
            
            new_line = f"{timestamp:.6f} lidar_{pos}/{idx:06d}.bin\n"
            f_new_data.write(new_line)
            idx += 1
            
        f_old_data.close()
        f_new_data.close()
        print(f"- {pos} lidar done")
        
def procSequence(root: Path, seq: str):
    calibration_fn = root / f'{seq}_calibration.tar.gz'
    data_fn = root / f'{seq}_data.tar.gz'
    las_fn = root / f'{seq}_las.tar.gz'
    pose_fn = root / f'{seq}_pose.tar.gz'
    extract_tar(root, calibration_fn)
    extract_tar(root, data_fn)
    extract_tar(root, las_fn)
    extract_tar(root, pose_fn)
    print(f"- done extraction")
    
    procCalib(root, seq, 'Vehicle2LeftVLP.txt', 'Base2LeftLiDAR.txt')
    procCalib(root, seq, 'Vehicle2RightVLP.txt', 'Base2RightLiDAR.txt')
    procCalib(root, seq, 'Vehicle2IMU.txt', 'Base2Imu.txt')
    
    procLidarScans(root, seq)

In [36]:
ROOT = Path('/data/datasets/dataset_kaist_urban')
SEQS = [
    # 'campus00', 
    'urban00', 
    'urban01', 
    'urban02', 
    'urban03', 
    'urban04'] 
# SEQS2 = [
#   'urban09', 
#   'urban10']
for seq in SEQS:
    print(f"Process {seq} starts ...")
    procSequence(ROOT, seq)
    
    

Process urban00 starts ...
- done extraction
-0.311890 0.394734 1.946610 -0.349085 0.152303 0.856673 0.347930
- done Base2LeftLiDAR.txt
-0.306052 -0.417145 1.952230 -0.349759 -0.146389 0.855857 -0.351777
- done Base2RightLiDAR.txt
-0.070000 0.000000 1.700000 0.000000 0.000000 0.000000 1.000000
- done Base2Imu.txt
- left lidar done
- right lidar done
Process urban01 starts ...
- done extraction
-0.311890 0.394734 1.946610 -0.349085 0.152303 0.856673 0.347930
- done Base2LeftLiDAR.txt
-0.306052 -0.417145 1.952230 -0.349759 -0.146389 0.855857 -0.351777
- done Base2RightLiDAR.txt
-0.070000 0.000000 1.700000 0.000000 0.000000 0.000000 1.000000
- done Base2Imu.txt
- left lidar done
- right lidar done
Process urban02 starts ...
- done extraction
-0.311890 0.394734 1.946610 -0.349085 0.152303 0.856673 0.347930
- done Base2LeftLiDAR.txt
-0.306052 -0.417145 1.952230 -0.349759 -0.146389 0.855857 -0.351777
- done Base2RightLiDAR.txt
-0.070000 0.000000 1.700000 0.000000 0.000000 0.000000 1.000000
-

After Extraction list below...
```bash
dataset_kaist_urban/
└── campus00
    ├── calibration
    ├── global_pose.csv
    ├── sensor_data
    └── sick_pointcloud.las

```